# $NH_3$ Test for different $\dot{m}_L$ values

In [ ]:
# Third-party module imports
import numpy as np # Scientific computing
import matplotlib.pyplot as plt # Visualisation
# CryoEvap module imports
from cryoevap.storage_tanks import Tank
from cryoevap.cryogens import Cryogen
import time

**Parámetros utilizados en la simulación**

- $\dot{Q}_{b} = 60\;[W]$: Calor de base que ingresa al fondo del tanque
- $d_i = 76.4\;[m]$: Diámetro interior del tanque
- $d_o = 80\;[m]$: Diámetro exterior del tanque
- $V_{tank} = 165000\;[m^3]$: Volumen del tanque.
- $T_{air} = 298.15\;[K]$: Temperatura del ambiente.
- $U_{L} = 0.19\;[\frac{W}{m^2K}]$: Coeficiente global de transferencia de calor para el líquido
- $U_{V} = 0.19\;[\frac{W}{m^2K}]$: Coeficiente global de transferencia de calor para el vapor
- $L_{F} = 0.55$: Llenado inicial de líquido
- $P = 116.325 \; [kPa]$: Presión dentro del tanque

In [2]:
# Step 2: Initialise tank object
Q_roof = 0 # Roof heat ingress / W
d_i = 76.4 # Internal diameter / m
d_o = 80   # External diameter / m
T_air = 298.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 0.19 # W/m2/K
U_V = 0.19 # W/m2/K

# Specify heat transfer rate at the bottom to prevent ground heating
Q_b = 60000 # W, 

# Vertically orientated cylindrical tank volume
V_tank = 165000 #m^3

# Initial liquid filling / Dimensionless
LF = 0.55 

# Specify tank operating pressure
P = 116325 # Pa

Para efectos demostrativos, se simulan los siguientes 8 flujos másicos:

1) $\dot{m}_{L_1} = -25\;[\frac{kg}{s}]$
2) $\dot{m}_{L_2} = -10\;[\frac{kg}{s}]$
3) $\dot{m}_{L_3} = -5\;[\frac{kg}{s}]$
4) $\dot{m}_{L_4} = -1\;[\frac{kg}{s}]$
5) $\dot{m}_{L_5} = 0\;[\frac{kg}{s}]$
6) $\dot{m}_{L_6} = 1\;[\frac{kg}{s}]$
7) $\dot{m}_{L_7} = 5\;[\frac{kg}{s}]$
8) $\dot{m}_{L_8} = 10\;[\frac{kg}{s}]$

In [3]:
m_L_values = np.array([-25, -10, -5, -1, 0, 1, 5, 10]) * 1e0  # Different mass flow rates to simulate

En este bloque se simula cada simulación

In [4]:
simulation_initial_time = time.time() # Start timer for the entire simulation set

# Initialise cryogen
ammonia = Cryogen(name = "ammonia") # Initialize cryogen object
ammonia.set_coolprops(P)

all_data = [] # Save the data from all simulations

for m_L in m_L_values:
    small_tank = Tank(d_i, d_o, V_tank, LF, m_L) # Initialize large-scale tank
    small_tank.set_HeatTransProps(U_L, U_V, T_air, Q_roof, Q_b, eta_w = 0.963)

    small_tank.cryogen = ammonia	# Set initialised cryogen as a tank property

    # Simulation setup
    dz = 0.2 # grid spacing / m
    n_z = 1 + int(np.round(small_tank.l_V/dz, 0)) # Number of nodes
    small_tank.z_grid = np.linspace(0, 1, n_z) # Set dimensionless grid
    small_tank.U_roof = 0 # Roof overall heat transfer coefficient W/m^2/K
    evap_time = 3600*628 # Define evaporation time / s
    small_tank.time_interval = 60 # Time-step to record data
    small_tank.plot_interval = evap_time/6 # Interval to plot vapour temperature profiles

    # Tiempo de la simulación
    initial_time = time.time() # Start timer
    print(f"Starting simulation for m_L = {m_L:.1f} kg/s")
    small_tank.evaporate(evap_time) # Simulate the evaporation
    elapsed_time = time.time() - initial_time
    current_simulation_time = time.time() - simulation_initial_time
    actual_minutes = current_simulation_time // 60
    actual_minutes_seconds = current_simulation_time % 60
    print(f"Simulation for m_L = {m_L:.1f} kg/s took {elapsed_time:.2f} seconds (Total elapsed time: {actual_minutes:.0f} minutes and {actual_minutes_seconds:.2f} seconds)\n")

    dict_info = small_tank.data
    all_data.append(dict_info)

Starting simulation for m_L = -25.0 kg/s
Simulation for m_L = -25.0 kg/s took 56.78 seconds (Total elapsed time: 0 minutes and 56.79 seconds)

Starting simulation for m_L = -10.0 kg/s
Simulation for m_L = -10.0 kg/s took 57.85 seconds (Total elapsed time: 1 minutes and 54.64 seconds)

Starting simulation for m_L = -5.0 kg/s
Simulation for m_L = -5.0 kg/s took 60.04 seconds (Total elapsed time: 2 minutes and 54.68 seconds)

Starting simulation for m_L = -1.0 kg/s
Simulation for m_L = -1.0 kg/s took 59.89 seconds (Total elapsed time: 3 minutes and 54.57 seconds)

Starting simulation for m_L = 0.0 kg/s
Simulation for m_L = 0.0 kg/s took 61.32 seconds (Total elapsed time: 4 minutes and 55.89 seconds)

Starting simulation for m_L = 1.0 kg/s
Simulation for m_L = 1.0 kg/s took 61.88 seconds (Total elapsed time: 5 minutes and 57.77 seconds)

Starting simulation for m_L = 5.0 kg/s
Simulation for m_L = 5.0 kg/s took 64.47 seconds (Total elapsed time: 7 minutes and 2.24 seconds)

Starting simulat

En este bloque se guardan los archivos de las simulaciones el archivos .txt para facilitar el post-procesamiento.

In [5]:
# Save all the dict info in a .txt file (one per mass flow rate)

for i, m_L in enumerate(m_L_values):
    filename = f'data/ammonia_tank_full_data_mL_{m_L:.1f}_kg_per_s.txt'
    with open(filename, 'w') as f:
        headers = []
        column_info = []

        for key, value in all_data[i].items():
            headers.append(key)
            column_info.append(value)

        # Save the internal diameter, external diameter, tank volume, and liquid fraction as constants
        headers.extend(['d_i (m)', 'd_o (m)', 'V_tank (m^3)', 'LF (-)'])
        column_info.extend([[d_i] * len(value), [d_o] * len(value), [V_tank] * len(value), [LF] * len(value)])  

        # column_info = np.transpose(column_info)
        header_str = ','.join(headers)
        np.savetxt(f, column_info, header=header_str, delimiter=', ')